In [ ]:
import pandas as pd
import os
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [1]:
filedir = '<folder containing the dataset here>'

In [ ]:
#df = pd.read_csv('data/landsat8_data_with_fire.csv')

df = pd.read_csv(os.path.join(filedir, 'dataset_path40_39_with_fire_v1.csv'))
                

df=df.drop(columns=["Unnamed: 0","Unnamed: 0.1"])
df.head(2)

In [ ]:
df_fire = df
df_fire['fire_nofire'] = df_fire['fire_nofire'].map(lambda x : 'no_fire' if x == 'no fire' else 'fire')
df_fire.head(2)
df = df_fire

In [ ]:
df_fire = df_fire[(df_fire.width == 512) & (df_fire.height == 512)]

In [ ]:
df_fire[df_fire.fire_nofire == 'fire'].shape

In [ ]:
df_dataset = df_fire[df_fire.fire_nofire == 'fire'].append(df_fire[df_fire.fire_nofire == 'no_fire'][0:9000])
#df_dataset = df_fire

In [ ]:
df_dataset.shape

In [ ]:
from PIL import TiffImagePlugin

def resize(image, width, height):
    #print(image)
    
    #im1 = Image.open(image)
    
    width = width
    height = height
    # use one of these filter options to resize the image
    im2 = image.resize((width, height), Image.NEAREST)      # use nearest neighbour
    return im2



In [ ]:

Path={}
for item in glob(os.path.join(filedir, 'ndvijpegs', '*.jpeg')):
   
    filename = os.path.splitext(os.path.basename(item))[0]
    #print(filename)
    Path[filename+".TIF"] = item
 



In [ ]:

df_dataset['path'] = df_dataset['filename'].map(Path.get)
df_dataset.head(2)

In [ ]:
df_dataset['image'] = df_dataset['path'].map(lambda x: np.asarray(Image.open(x).resize((28,28))))

In [ ]:
df_dataset.head(2)


In [ ]:
df_dataset['image'] = df_dataset['image'].map(lambda x: np.array(x).reshape(x.shape[0],x.shape[1]))
#df_dataset['image'][0].reshape(512,512,1)

In [ ]:
X=df_dataset.drop(columns=["fire_nofire"])
Y=df_dataset["fire_nofire"]
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
     X, Y, test_size=0.33, random_state=42)




In [ ]:
x_train = np.asarray(X_train['image'].tolist())

x_test = np.asarray(X_test['image'].tolist())
#x_train = x_train.reshape(x_train.shape[0], *(512,512,1))
#x_test = x_test.reshape(x_test.shape[0], *(75, 100, 3))
xTrainmean = np.mean(x_train)
xTrainstd = np.std(x_train)
xTestmean = np.mean(x_test)
xTeststd = np.std(x_test)
x_train = (x_train - xTrainmean)/xTrainstd
x_test = (x_test - xTestmean)/xTeststd


In [ ]:
import keras

In [ ]:
#import sys
#!{sys.executable}  -m pip install --user keras

In [ ]:
#!{sys.executable}  -m pip install --user tensorflow

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

t1 = le.fit_transform(Y_train)
t2 = le.fit_transform(Y_test)
y_train = keras.utils.to_categorical(t1, num_classes=2)
y_test = keras.utils.to_categorical(t2, num_classes=2)
y_train.shape

In [ ]:
nsamples, nx, ny = x_test.shape
d2_test_dataset = x_test.reshape((nsamples,nx*ny))
d2_test_dataset.shape

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM


In [ ]:
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size = 0.1, random_state = 2)
x_train = x_train.reshape(x_train.shape[0], *(28, 28, 1))
x_test = x_test.reshape(x_test.shape[0], *(28, 28, 1))

x_validate = x_validate.reshape(x_validate.shape[0], *(28, 28, 1))

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, MaxPooling2D
import itertools
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

dimensions = (28, 28,1)
Classes = 2

model = Sequential()

model.add(Conv2D(128, (2, 2),activation='relu',padding = 'Same',input_shape=dimensions))
#model.add(Conv2D(128, (3, 3), activation='softmax',padding = 'Same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.40))
#model.add(Dense(Classes, activation='softmax'))

model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
#model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.40))

model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
#model.add(Conv2D(32, (3, 3), activation='relu',padding = 'Same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.40))


model.add(Conv2D(32, (3, 3), activation='relu',padding = 'Same'))
#model.add(Conv2D(16, (3, 3), activation='relu',padding = 'Same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(Classes, activation='softmax'))

#model.add(Dropout(rate=0.5))

model.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy', 'categorical_crossentropy'],
)

history = model.fit(x_train,
          y_train,
          validation_data=(x_validate, y_validate),
          batch_size=100,
          epochs=500,
          )


In [ ]:
print(model.summary())

In [ ]:
from sklearn.metrics import classification_report
import sklearn.metrics as metrics

#from keras import metrics


y_pred = model.predict(x_test.astype('float64'))
target_names=['fire', 'no_fire']
print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1), target_names=target_names))


matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))


In [ ]:
from sklearn.metrics import roc_curve
y_pred_keras = model.predict(x_test)
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test.argmax(axis=1), y_pred_keras.round().argmax(axis=1))
fpr_keras_train, tpr_keras_train, thresholds_keras_train = roc_curve(y_train.argmax(axis=1), y_train.round().argmax(axis=1))

from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
#plt.plot(fpr_keras_train, tpr_keras_train, label='Keras (area = {:.3f})'.format(auc_keras))
#plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()
# Zoom in view of the upper left corner.
plt.figure(2)
plt.xlim(0, 0.2)
plt.ylim(0.8, 1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
#plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve (zoomed in at top left)')
plt.legend(loc='best')
plt.show()

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
def draw_confusion_matrix(true,preds):
    conf_matx = confusion_matrix(true, preds)
    sns.heatmap(conf_matx, annot=True,annot_kws={"size": 12},fmt='g')
    plt.show()
draw_confusion_matrix(y_test.argmax(axis=1), y_pred.round().argmax(axis=1))
#draw_confusion_matrix(y1, y2)


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

In [ ]:
print(history.history.keys())
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()


In [ ]:
model.save('NDVI_CNN.h5')


In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
def draw_confusion_matrix(true,preds):
    conf_matx = confusion_matrix(true, preds)
    sns.heatmap(conf_matx, annot=True,annot_kws={"size": 12},fmt='g', cbar=False, cmap="viridis")
    plt.show()
draw_confusion_matrix(test_y.argmax(axis=1), y_pred.round().argmax(axis=1))


-------------------------------------------------